In [ ]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import random

In [ ]:
def get_output(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": message,
            }
        ],
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content

In [ ]:
def get_bias_output(fact):
    template="""
Please answer the question.

Question:%s
A.no   B.yes

Output format:
Ans:  (Only one option can be selected)

"""%(fact)
    return template

In [ ]:
jishu1=250
jishu2=250
parquet_file = pq.ParquetFile('./data/test.parquet')
data = parquet_file.read().to_pandas()

In [ ]:
print(data['question'][0])
print(data['context'][0])

In [ ]:
i=0
f=open('./data/test.jsonl','a')
while jishu1>0 or jishu2>0:
    if 'no' in data['answer'][i] and jishu1>0:
        now=get_output(get_bias_output(data['question'][i]))
        try:
            now=re.findall(r"Ans([\S|\s]*)",now, re.S|re.I)[0]
        except:
            a=1
        if 'no' in now or 'A' in now or 'No' in now:
            jishu1=jishu1-1
            input_dict={'number':i,'question':data['question'][i],'ans':'A','context':data['context'][i]}
            input_json=json.dumps(input_dict)
            f.write(input_json+'\n')
    elif 'yes' in data['answer'][i] and jishu2>0:
        now=get_output(get_bias_output(data['question'][i]))
        try:
            now=re.findall(r"Ans([\S|\s]*)",now, re.S|re.I)[0]
        except:
            a=1
        if 'yes' in now or 'B' in now or 'Yes' in now:
            jishu2=jishu2-1
            input_dict={'number':i,'question':data['question'][i],'ans':'B','context':data['context'][i]}
            input_json=json.dumps(input_dict)
            f.write(input_json+'\n')
    i=i+1

In [ ]:
f.close()

In [ ]:
print(jishu1,jishu2)